# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

df = pd.read_csv('/datasets/data.csv')
df.info()
display(df.isna().sum())#количество имеющихся пропусков во всём фрейме
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

Данные представлены 12 столбцами и 21525 строками:
- children — количество детей в семье (int64)
- days_employed — общий трудовой стаж в днях (float64)
- dob_years — возраст клиента в годах (int64)
- education — уровень образования клиента (object)
- education_id — идентификатор уровня образования (int64)
- family_status — семейное положение (object)
- family_status_id — идентификатор семейного положения (int64)
- gender — пол клиента (object)
- income_type — тип занятости (object)
- debt — имел ли задолженность по возврату кредитов: либо True, либо False (int64)
- total_income — ежемесячный доход (float64)
- purpose — цель получения кредита (object)

В столбцах *days_employed* и *total_income* наблюдаю пропуски, причём значение количества для этих столбцов равно и составляет - 19351 из 21525. Возможно имеется какая-то связь возникновения таких пропусков. К тому же в столбце *days_employed* наблюдаются отрицательные значения. Данное поведение буду расценивать пока как артефакт, к которому обращусь на одном из шагов обработки данных.

В стобце *education* различный регистр записи - приведу к единому отображению данных.

Так как согласно основной гипотезе исследования нужно разобраться, влияет ли семейное положение и количество детей у клиента на факт погашения кредита в срок, то основными показателями для вывода по гипотезе буду считать семейное положение, количество детей клиента, ежемесячный доход и наличие задолженности у клиента.

## Шаг 2. Предобработка данных

Прежде чем приступить к обработке пропусков поясню логику действий.
Имеется два столбца с пропусками *days_employed* и *total_income* оба они являются количественными, а значит обработка должна вестись как с количественными переменными. В работу пока возьму *total_income*. Здесь для замены пропусков корректней всего взять медиану по ежемесячному доходу, однако есть один момент на который стоит обратить внимание, медиану, как я считаю и буду использовать, я буду брать по типу *income_type*, т.е.  по типу занятости клиента, а не по всему стобцу *total_income*. В рамках данного исследования считаю это наиболее обоснованным.

### Обработка пропусков

In [2]:
df['income_type'].unique() #узнал и проверил уникальные значения для типов занятости клиентов

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [3]:
#медианное значение ежемесячного дохода для каждой категории в столбце total_income 
median_income_type = df.groupby('income_type')['total_income'].transform('median')
print(median_income_type)

0        142594.396847
1        142594.396847
2        142594.396847
3        142594.396847
4        118514.486412
             ...      
21520    172357.950966
21521    118514.486412
21522    142594.396847
21523    142594.396847
21524    142594.396847
Name: total_income, Length: 21525, dtype: float64


In [4]:
#заменю все пропущенные значения в столбце total_income на медианные значения для каждого типа занятости
df['total_income'] = df['total_income'].fillna(median_income_type)
#проверю на наличие пропущенных значений по данному столбцу
display(df.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

In [5]:
#какой категории занятости клиентов больше всех принадлежит количество записей о трудовом стаже
display(df.groupby('income_type')['days_employed'].count())
#какое суммарное количество отработанных дней у каждой категории (делаю для выявления артефакта отображения)
display(df.groupby('income_type')['days_employed'].sum())

income_type
безработный            2
в декрете              1
госслужащий         1312
компаньон           4577
пенсионер           3443
предприниматель        1
сотрудник          10014
студент                1
Name: days_employed, dtype: int64

income_type
безработный        7.328273e+05
в декрете         -3.296760e+03
госслужащий       -4.460665e+06
компаньон         -9.664447e+06
пенсионер          1.256707e+09
предприниматель   -5.208481e+02
сотрудник         -2.329756e+07
студент           -5.787516e+02
Name: days_employed, dtype: float64

Исходя из полученных данных у двух безработных сумма трудовых дней превышает остальные данные, а у сотрудников (самой большой категории по типу занятости) количество отработанных дней в большом минусе. Могу сделать вывод, что при получении данных могла возникнуть как технологическая ошибка. Допустим в анкете стоит обратный отсчёт, либо ошибка возникла в случае человеческого фактора. Но в любом случае с этой категорией данных работать нет необходимости. Данные из этого стобца не влияют на гипотезу, однако в реальном случае необходимо было бы проверить как принимаются эти данные по этому столбцу (сообщить коллегам о возможной проблеме). В рамках проекта я удалю данный столбец.

In [6]:
#удаляю столбец days_employed
del df['days_employed']

In [7]:
#проверю произошло ли корректное исправление пропусков и удаление столбца
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
children            21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(5), object(5)
memory usage: 1.8+ MB


Столбца нет, пропусков тоже нет.

**Вывод**

После анализа данных было выявлено, что имеется два столбца с пропусками days_employed и total_income. Для дальнейшей корректной работы было принято решение, что в столбце total_income необходимо заменить пропуски на медианное значение в рамках категории занятости клиентов. А в столбце days_employed имеются артефакты, которые можно считать как ошибками технологическими и/или вследствие допущения человеческого фактора, для дальнейшей работы такой столбец не актуален и был удалён из таблицы.

### Замена типа данных

In [8]:
#переведу total_income из вещевстенного в целочисленное значение

df['total_income'] = df['total_income'].astype('int64')
df.info()
#display(df.head(30))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
children            21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(6), object(5)
memory usage: 1.8+ MB


**Вывод**

В данном случае понадобилась лишь одна замена типа данных для визуальной чистоты. Все строковые значения должны оставаться в строковом значении, значения в столбце *total_income* были переведены в целочисленные. Использовал метод *astype('int64')*

### Обработка дубликатов

In [9]:
#проверю на наличие дубликатов во всей таблице
df.duplicated().sum()

54

In [10]:
#проверю на наличие дубликатов в столбце education можно и через Counter(df['education']), но визуально так лучше
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [11]:
#приведу все значения столбца education к единому регистру
df['education'] = df['education'].str.lower()

In [12]:
#проверка
df['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [13]:
#проверю на наличие дубликатов в столбце family_status
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [14]:
#дубликатов нет, но приведу все значения столбца family_status к единому регистру
df['family_status'] = df['family_status'].str.lower()

In [15]:
#проверка
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [16]:
#проверю на наличие дубликатов в столбце gender
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [17]:
#дубликатов нет, но присутствует некатегорийное значение - удалю 
df = df.drop(df[df['gender']=='XNA'].index)

In [18]:
#проверка
df['gender'].value_counts()

F    14236
M     7288
Name: gender, dtype: int64

In [19]:
#проверю на наличие дубликатов в столбце children
df['children'].value_counts()

 0     14148
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [20]:
#были выявлены не дубликаты, а возможно опечатки/ошибки при вводе значений
#заменю -1 на 1, а 20 на 2
df['children'] = df['children'].replace(-1, 1)
df['children'] = df['children'].replace(20, 2)

In [21]:
#проверка
df['children'].value_counts()

0    14148
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [22]:
#осталось проверить столбец dob_years
df['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    263
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [23]:
#такая простая проверка позволила выявить некорректность в данных
#есть клиенты с возрастом 0 и их 101 человек, явно ошибка при заполнении
#общее количество клиентов в данной выборке 928317 поэтому я могу пренебречь таким маленьким процентом 
#и просто удалю это значение
df = df.drop(df[df['dob_years'] == 0].index)

In [24]:
#проверю, осталось ли это значение
df['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 59, 29, 60, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75])

In [25]:
#удалю все возможные дубликаты
df = df.drop_duplicates().reset_index(drop = True)

In [26]:
#проверю на наличие дубликатов во всей таблице
df.duplicated().sum()

0

**Вывод**

Все дубликаты удалены. Найденные артефакты исправлены, скорректированы возможные технологические ошибки или ошибки возникшие вследствие человеческого фактора. Значения столбцов *education* и *family_status* приведены к единому нижнему регистру.

### Лемматизация

In [27]:
#присвою m библиотеку pymystem3 для лемматизации
m = Mystem() 

In [28]:
#по столбцу purpose узнаю все цели
df['purpose'].value_counts()

свадьба                                   786
на проведение свадьбы                     764
сыграть свадьбу                           760
операции с недвижимостью                  672
покупка коммерческой недвижимости         658
покупка жилья для сдачи                   649
операции с коммерческой недвижимостью     648
операции с жильем                         646
покупка жилья                             640
жилье                                     640
покупка жилья для семьи                   637
строительство собственной недвижимости    633
недвижимость                              629
операции со своей недвижимостью           627
строительство жилой недвижимости          621
строительство недвижимости                619
покупка своего жилья                      619
покупка недвижимости                      617
ремонт жилью                              605
покупка жилой недвижимости                603
на покупку своего автомобиля              502
заняться высшим образованием      

Из списка указанных целей могу выделить несколько основных целей:
- свадьба;
- недвижимость и имеется второе значение - жильё;
- авто;
- образование.

In [29]:
#увидел в обсуждениях такой вариант, решил попробовать, но пока мой вариант (который выше) мне нравится больше,
#но перепроверку провёл
lemmas_list = []
unique_purpose = df['purpose']
m = Mystem()
for purpose in unique_purpose:
    try:
        lemmas = ''.join(m.lemmatize(purpose)).strip()
        lemmas_list.append(lemmas)
    except:
        print('Ошибка прохождения по целям')
print(Counter(lemmas_list))

Counter({'автомобиль': 966, 'свадьба': 786, 'на проведение свадьба': 764, 'сыграть свадьба': 760, 'операция с недвижимость': 672, 'покупка коммерческий недвижимость': 658, 'покупка жилье для сдача': 649, 'операция с коммерческий недвижимость': 648, 'операция с жилье': 646, 'покупка жилье': 640, 'жилье': 640, 'покупка жилье для семья': 637, 'строительство собственный недвижимость': 633, 'недвижимость': 629, 'операция со свой недвижимость': 627, 'строительство жилой недвижимость': 621, 'строительство недвижимость': 619, 'покупка свой жилье': 619, 'покупка недвижимость': 617, 'ремонт жилье': 605, 'покупка жилой недвижимость': 603, 'на покупка свой автомобиль': 502, 'заниматься высокий образование': 493, 'сделка с подержанный автомобиль': 484, 'на покупка подержать автомобиль': 478, 'свой автомобиль': 475, 'на покупка автомобиль': 466, 'приобретение автомобиль': 459, 'дополнительный образование': 458, 'сделка с автомобиль': 454, 'высокий образование': 449, 'образование': 444, 'получение до

In [30]:
#создам функцию. В ней будет переменная с леммитизированными значениями из столбца purpose,
#которая будет проверять переменную функции и закидывать к какой цели можно определить

def lemmatization(purpose):
    lemmas = m.lemmatize(purpose)
    lemma_name = []
    try:
        if ('жилье' in lemmas) or ('недвижимость' in lemmas):
            lemma_name = 'недвижимость'
            return lemma_name
        if 'автомобиль' in lemmas:
            lemma_name = 'автомобиль'
            return lemma_name
        if 'образование' in lemmas:
            lemma_name = 'образование'
            return lemma_name
        if 'свадьба' in lemmas:
            lemma_name = 'свадьба'
            return lemma_name
    except:
        print('ошибка выбора цели')

In [31]:
#объявлю новую переменную в которой соберу все значения функции
lemma_purpose = df['purpose'].apply(lemmatization)
#заменю прежние значения на 4 основные цели
df['purpose'] = lemma_purpose

In [32]:
df.head(10)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба
5,0,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,недвижимость
6,0,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,недвижимость
7,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,свадьба
9,0,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,недвижимость


**Вывод**

После лемматизации цели обрели структурность и теперь стало ясно, что основных целей всего 4. Таблицу теперь проще анализировать.

### Категоризация данных

 Итак, нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.
 
 Поэтому я для категоризации буду использовать "словарь" в котором как раз будет обозначена возможная зависимость между семейным положением и количеством детей клиента на факт погашения кредита в срок. Словарь будет из столбцов *family_status*, *children* и *debt*. И заодно произведу классификацию по уровню дохода клиентов.

In [33]:
#добавлю функцию для категоризации по наличию детей
def category_children(child):
    try:
        if child == 0:
            return 'нет детей'
        if child > 0:
            return 'есть дети'
    except:
        print('Ошибка в заполнении поля "Дети"')

In [35]:
#создаю словарь
credits_dict = df[['category_children', 'family_status', 'debt']]

In [36]:
#Теперь сгруппирую и отсортирую по задолженности от тех у кого есть до тех у кого нет
display(credits_dict.sort_values(by='debt', ascending=False).head(10))

,category_children,family_status,debt
11110,нет детей,в разводе,1
13234,нет детей,не женат / не замужем,1
8764,есть дети,женат / замужем,1
18773,есть дети,не женат / не замужем,1
8758,есть дети,женат / замужем,1
8754,нет детей,женат / замужем,1
8752,есть дети,гражданский брак,1
13250,нет детей,женат / замужем,1
18756,нет детей,не женат / не замужем,1
13257,есть дети,гражданский брак,1


Для категоризации уровня дохода введу 4 уровня дохода, ориентируясь на статью, в которой были обозначены данные от Федеральной службы государственной статистики. Служба разделяет уровень жизни россиян в зависимости от доходов на следующие категории:
- крайняя нищета — доходы ниже прожиточного минимума (до 7-8 тыс.р.)
- нищета — доходы от одного до двух прожиточных минимума (от 8 до 12 тыс.р.)
- бедность — доходы от 12 до 20 тысяч рублей в месяц.
- выше бедности — доходы от 20 до 30 тысяч рублей в месяц.
- средний достаток — доходы от 30 до 60 тысяч рублей в месяц.
- состоятельные — доходы от 60 до 90 тысяч рублей в месяц.
- богатые — доходы от 90 тысяч рублей в месяц.
- сверхбогатые — доходы свыше 150 тысяч рублей в месяц.

В своей категоризации я буду использовать следующие категории:
- низкий доход - 29 999 рублей в месяц.
- средний доход - от 30 000 до 59 999 рублей в месяц.
- высокий доход - от 60 000 до 149 999 рублей в месяц.
- сверхвысокий доход - свыше 150 тысяч рублей в месяц.

In [37]:
#добавлю функцию для категоризации по доходу
def total_income_sort(income):
    try:
        if income < 29999:
            return 'низкий доход'
        if 30000 < income < 59999:
            return 'средний доход'
        if 60000 < income < 149999:
            return 'высокий доход'
        if income > 150000:
            return 'сверхвысокий доход'
    except:
        print('Ошибка в доходах')

In [38]:
#проверка
total_income_sort(90000)

'высокий доход'

In [39]:
#добавлю новый столбец с этой категоризацией
df['total_income_sort'] = df['total_income'].apply(total_income_sort)
df.head(10)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,category_children,total_income_sort
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,есть дети,сверхвысокий доход
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,есть дети,высокий доход
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,нет детей,высокий доход
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,есть дети,сверхвысокий доход
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,нет детей,сверхвысокий доход
5,0,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,недвижимость,нет детей,сверхвысокий доход
6,0,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,недвижимость,нет детей,сверхвысокий доход
7,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,нет детей,высокий доход
8,2,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,свадьба,есть дети,высокий доход
9,0,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,недвижимость,нет детей,высокий доход


In [40]:
#добавлю функцию для категоризации по возрасту
def dob_years_group(age):
    try:
        if 18 <= age <= 35:
            return 'молодые'
        if 36 <= age <= 64:
            return 'взрослые'
        if 65 < age:
            return 'пенсионеры'
    except:
        print('Ошибка заполнения возраста')

In [41]:
#проверка
dob_years_group(19)

'молодые'

In [42]:
#добавлю новый столбец с этой категоризацией
df['dob_years_group'] = df['dob_years'].apply(dob_years_group)
df.head(10)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,category_children,total_income_sort,dob_years_group
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,есть дети,сверхвысокий доход,взрослые
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,есть дети,высокий доход,взрослые
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,нет детей,высокий доход,молодые
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,есть дети,сверхвысокий доход,молодые
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,нет детей,сверхвысокий доход,взрослые
5,0,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,недвижимость,нет детей,сверхвысокий доход,молодые
6,0,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,недвижимость,нет детей,сверхвысокий доход,взрослые
7,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,нет детей,высокий доход,взрослые
8,2,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,свадьба,есть дети,высокий доход,молодые
9,0,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,недвижимость,нет детей,высокий доход,взрослые


**Вывод**

Для категоризации я создал один "словарь" по сортировке которого можно попытаться найти зависимость между семейным положением и количеством детей клиента на факт погашения кредита в срок.

Так же были написаны функции, целью которых является создание категорий клиентов по их доходам, возрасту и по наличию детей. Это позволит ответить на все вопросы зависимости между уровнем дохода, семейным положением, наличием детей и возвратом кредита в срок.
Во время лемматизации так же была проведена категоризация клиентов на 4 основные цели, данное действие было произведено мною с целью попытки ответить на вопрос как разные цели кредита влияют на его возврат в срок?
Как раз ответы на эти вопросы будут представлены ниже.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [43]:
#чтобы ответить на вопрос про есть ли зависимость между наличием детей и возвратом кредита в срок?
#добавлю сводную таблицу по общему количеству задолженностей у клиентов у которых нет детей и у тех, у которых есть дети
df_pivot_children = df.pivot_table(index=['children'], columns='category_children', values='debt', aggfunc='sum')

In [44]:
df_pivot_children.head(10)

category_children,есть дети,нет детей
children,,
0,NaN,1058.0
1,442.0,NaN
2,202.0,NaN
3,27.0,NaN
4,4.0,NaN
5,0.0,NaN


Как видно из сводной таблицы количество клиентов у которых нет детей и есть задолженность составляет - 1063 человека.
В это же время мы наблюдаем, что чем больше детей у клиентов, тем меньше клиентов у которых есть задолженность.
Но данная таблица может не отображать все результаты, поэтому для проверки добавлю ещё и агрегированную функцию.

In [45]:
#добавлю агрегированную функцию, которая считает по столбцу debt (задолженность)
#общее количество, сумму и находит среднее, что позволит сделать выводы как наличие детей влияет на задолженность
children_debt = df.groupby('children').agg({'debt':['count','sum', 'mean']})

In [46]:
children_debt

debt                
          count   sum      mean
children                       
0         14021  1058  0.075458
1          4839   442  0.091341
2          2114   202  0.095553
3           328    27  0.082317
4            41     4  0.097561
5             9     0  0.000000

Благодаря агрегированной функции наблюдается изменение в интерпретации результатов. Согласно этим расчётам, несмотря на то, что задолженности больше у клиентов, у которых нет детей, при этом этих клиентов больше всего у банка и если рассмотреть соотношение количества клиентов и количества задолженностей, то получается, что среди клиентов у которых нет детей самый низкий процент задолженностей.

У клиентов у которых 4 детей самый высокий процент задолженностей. У клиентов с одним, двумя детьми процент у кого есть задолженность выше, чем среди клиентов у которых нет детей. 

In [47]:
#добавлю сводную таблицу по соотношению количества клиентов с задолженностью
#от общего количества клиентов по каждой категории наличия детей
df_pivot_children = df.pivot_table(index=['children'], columns='category_children', values='debt', aggfunc='mean')

In [48]:
display(df_pivot_children.head(10))

category_children,есть дети,нет детей
children,,
0,NaN,0.075458
1,0.091341,NaN
2,0.095553,NaN
3,0.082317,NaN
4,0.097561,NaN
5,0.000000,NaN


**Вывод**

Как видим из полученных результатов зависимость между наличием детей и возвратом кредита в срок имеется.

Нет совсем задолженности у клиентов, у которых 5 детей. Здесь правда присутствует один момент - малая выборка, чтобы делать окончательный вывод, однако клиенты, с таким количеством детей, подходят к взятию кредита и исполнению финансовых обязательств с большей долей ответственности. При этом самый низкий процент задолженностей у клиентов у которых нет детей, что говорит о возможности свободных средств для погашения кредита во время.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [49]:
#добавлю сводную таблицу для того чтобы сделать выводы
#сводная таблица группируется по family_status, значения, которые отображены это столбец debt
df_pivot_family_status = df.pivot_table(index=['family_status'],
                                        columns='family_status_id', values='debt', aggfunc='sum')

In [50]:
df_pivot_family_status.head(10)

family_status_id,0,1,2,3,4
family_status,,,,,
в разводе,NaN,NaN,NaN,85.0,NaN
вдовец / вдова,NaN,NaN,62.0,NaN,NaN
гражданский брак,NaN,386.0,NaN,NaN,NaN
женат / замужем,927.0,NaN,NaN,NaN,NaN
не женат / не замужем,NaN,NaN,NaN,NaN,273.0


Как видно из таблицы больше всего задолежнностей у клиентов, находящихся в семейном статусе женат/замужем.

In [51]:
#проверю показатели по соотношению между количеством задолженностей
#и общим количеством должников в разных семейных статусах
df_pivot_family_status = df.pivot_table(index=['family_status'],
                                        columns='family_status_id', values='debt', aggfunc='mean')

In [52]:
df_pivot_family_status.head(10)

family_status_id,0,1,2,3,4
family_status,,,,,
в разводе,NaN,NaN,NaN,0.07173,NaN
вдовец / вдова,NaN,NaN,0.06499,NaN,NaN
гражданский брак,NaN,0.093485,NaN,NaN,NaN
женат / замужем,0.075427,NaN,NaN,NaN,NaN
не женат / не замужем,NaN,NaN,NaN,NaN,0.097709


In [53]:
#добавлю агрегированную функцию для проверки и корректности выводов
family_grouped = df.groupby('family_status').agg({'debt':['count','sum', 'mean']}) 

In [54]:
family_grouped

debt               
                       count  sum      mean
family_status                              
в разводе               1185   85  0.071730
вдовец / вдова           954   62  0.064990
гражданский брак        4129  386  0.093485
женат / замужем        12290  927  0.075427
не женат / не замужем   2794  273  0.097709

In [55]:
#перед выводом добавлю ещё одну сводную таблицу
#которая как мне кажется подробнее отображает значимые показатели
df_pivot_children_family_status = df.pivot_table(index=['children'],
                                                 columns='family_status', values='debt', aggfunc='mean')

In [56]:
df_pivot_children_family_status

family_status,в разводе,вдовец / вдова,гражданский брак,женат / замужем,не женат / не замужем
children,,,,,
0,0.070785,0.061758,0.083579,0.069114,0.093375
1,0.066667,0.086420,0.118712,0.081748,0.112832
2,0.098765,0.125000,0.093484,0.094088,0.120482
3,0.090909,0.000000,0.142857,0.068826,0.125000
4,0.000000,0.000000,0.000000,0.103448,0.500000
5,NaN,NaN,0.000000,0.000000,NaN


**Вывод**

Зависимость между семейным положением и возвратом кредита в срок наблюдается. Клиенты со статусами вдовец/вдова, в разводе и женат/замужем имеют низкий процент количества задолженностей среди общего числа клиентов с этими же статусами. При этом высокий процент задолженностей наблюдается у клиентов, которые в семейных статусах указывают гражданский брак, не женат/не замужем. При этом основные клиенты банка это люди с семейным статусом женат/замужем, но процент задолженностей у них средний среди остальных клиентов.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [57]:
#добавлю сводную таблицу для того чтобы сделать выводы
#сводная таблица группируется по total_income_sort, значения, которые отображены это столбец debt - задолженность
df_pivot_total_income = df.pivot_table(index=['total_income_sort'],
                                        columns='dob_years_group', values='debt', aggfunc='sum')

In [58]:
df_pivot_total_income.head(10)

dob_years_group,взрослые,молодые,пенсионеры
total_income_sort,,,
высокий доход,508,389,20
низкий доход,2,0,0
сверхвысокий доход,449,295,11
средний доход,27,14,4


In [59]:
#добавлю сводную таблицу для того чтобы сделать выводы
#сводная таблица группируется по total_income_sort, значения, которые отображены это столбец debt - задолженность
#указано в соотношении наличия задолженности на общее количество клиентов в каждой категории
df_pivot_total_income = df.pivot_table(index=['total_income_sort'],
                                        columns='dob_years_group', values='debt', aggfunc='mean')

In [60]:
df_pivot_total_income.head(10)

dob_years_group,взрослые,молодые,пенсионеры
total_income_sort,,,
высокий доход,0.073442,0.117629,0.047170
низкий доход,0.117647,0.000000,0.000000
сверхвысокий доход,0.070101,0.094703,0.055556
средний доход,0.050373,0.088050,0.052632


In [61]:
#добавлю агрегированную функцию
total_income_grouped = df.groupby('total_income_sort').agg({'debt':['count','sum', 'mean']}) 

In [62]:
total_income_grouped

debt               
                    count  sum      mean
total_income_sort                       
высокий доход       10768  925  0.085903
низкий доход           22    2  0.090909
сверхвысокий доход   9782  759  0.077591
средний доход         780   47  0.060256

**Вывод**

Как видно из таблиц клиенты со средним доходом имеют самый низкий процент задолженностей, самый высокий у клиентов с низким доходом. Клиентов со сверхвысоким доходом большого количество у банка при этом один из самых низких процентов задолженностей.

- Как разные цели кредита влияют на его возврат в срок?

In [63]:
#добавлю сводную таблицу
#сводная таблица группируется по purpose, значения, которые отображены это столбец debt - задолженность
df_pivot_purpose = df.pivot_table(index=['purpose'],
                                        columns='income_type', values='debt', aggfunc='sum')

In [64]:
df_pivot_purpose.head(10)

income_type,безработный,в декрете,госслужащий,компаньон,пенсионер,предприниматель,сотрудник,студент
purpose,,,,,,,,
автомобиль,NaN,1.0,22.0,84.0,50.0,NaN,243.0,NaN
недвижимость,1.0,NaN,36.0,167.0,92.0,0.0,483.0,0.0
образование,NaN,NaN,21.0,72.0,48.0,NaN,229.0,NaN
свадьба,NaN,NaN,7.0,52.0,25.0,0.0,100.0,NaN


Данная сводная таблица позволяет не просто наблюдать какие задолженность по целям кредита, но и у какой категории по типу занятости.

In [65]:
#добавлю сводную таблицу
#сводная таблица группируется по purpose, значения, которые отображены это столбец debt - задолженность
#указано в соотношении наличия задолженности на общее количество клиентов в каждой категории
df_pivot_purpose = df.pivot_table(index=['purpose'],
                                        columns='income_type', values='debt', aggfunc='mean')

In [66]:
df_pivot_purpose.head(10)

income_type,безработный,в декрете,госслужащий,компаньон,пенсионер,предприниматель,сотрудник,студент
purpose,,,,,,,,
автомобиль,NaN,1.0,0.077193,0.080383,0.063131,NaN,0.112448,NaN
недвижимость,0.5,NaN,0.047809,0.065800,0.048910,0.0,0.086451,0.0
образование,NaN,NaN,0.082031,0.075710,0.067133,NaN,0.110468,NaN
свадьба,NaN,NaN,0.044586,0.099426,0.059382,0.0,0.082781,NaN


In [67]:
#добавлю агрегированную функцию
purpose_grouped = df.groupby('purpose').agg({'debt':['count','sum', 'mean']}) 

In [68]:
purpose_grouped

debt               
              count  sum      mean
purpose                           
автомобиль     4284  400  0.093371
недвижимость  10763  779  0.072378
образование    3995  370  0.092616
свадьба        2310  184  0.079654

**Вывод**

Клиенты у которых цель кредита был автомобиль или образование имеют самые высокие показатели по задолженности. Люди, которые брали кредит на жильё имеют самый низкий процент задолженности среди представленных клиентов. Здесь можно судить о более серьёзном подходе к такой категории как недвижимость. 

## Шаг 4. Общий вывод

Заказчиком исследования является кредитный отдел банка. Была поставлена задача, узнать влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.
В ходе работе с предоставленными данными о платёжеспособности клиента были проанализированы разные категории клиентов в разных разрезах: цели кредита, уровень дохода, возраст, наличие детей и семейное положение.

Самое низкое соотношение количества должников к общему количеству клиентов по конкретной категории имеют клиенты в семейных статусах: вдовец/вдова - 0.065693, в разводе - 0.071130, женат/замужем - 0.075452.

По наличию и отсутствию детей можно выделить клиентов, у которых самое низкое соотношение количества должников к общему количеству клиентов по конкретной категории: 5 детей - 0 задолженностей, нет детей - 0.075444, 3 детей - 0.081818.
